### Import package

In [1]:
import pandas as pd
import pyqsar
from pyqsar import data_tools as dt  #widget for data
from sklearn.preprocessing import MinMaxScaler #Standardisation of data
from pyqsar import clustering as cl #Feature clustering for feature selection
from pyqsar import fssml #feature selection code of modified version
from pyqsar import export_model as em 

### Import Data

In [ ]:
data = pd.read_csv('../1-3Dmd.csv',sep=",")
X_data = data.iloc[:,1:-2]
y_data = data.iloc[:,-1:]

In [ ]:
print(X_data.shape)
print(y_data.shape)

### Data Cleaning

In [ ]:
X_data = dt.rm_empty_feature(X_data)
X_data = dt.rmNaN(X_data)

#Standardisation of data:
header = list(X_data.columns.values)
scaler = MinMaxScaler()
X_data_scaled = scaler.fit_transform(X_data)
X_data = pd.DataFrame(X_data_scaled, columns=header)
X_data.head()

In [ ]:
print("The shape of X data after treated is ",X_data.shape)

### Cluster analysis

In [ ]:
cl.cophenetic(X_data)

In [ ]:
clust = cl.FeatureCluster(X_data, 'average', 3) #choose the highest score method
clust_info = clust.set_cluster()

In [ ]:
clust.cluster_dist()

### Feature screening by using Machine learning

In [ ]:
def exportfeature(feature_set,outputfilename):
    mymodel = em.ModelExport(X_data,y_data, feature_set)
    mymodel.to_csv(outputfilename+ +"selection.csv",sep=',',index=False,header=True)
    return None

### Export Feature as csv

In [ ]:
score_group = ""
index_group = ""

for f in range(1,41):
    select = fssml.selection(X_data, y_data,
                       clust_info,
                       model='regression',
                       learning=10000,
                       bank=200,
                       component=f)
    
    feature_set = select[0] #a list
    score_group += str(select[0])+"," #a scored string
    index_group += str(f)+","
    outputfilename='.\\f'+ str(f)
    exportfeature(feature_set,outputfilename) #export feature's data as csv

with open("score.csv","w") as f:
    f.write("feature selection number:,")
    f.write(index_group)
    f.write("\n")
    
    f.write("feature selection score:,")
    f.write(score_group)
    f.close()